# RL Training Examples

Simple examples of training RL algorithms with Ray RLlib.

**Note**: For comprehensive hyperparameter documentation, see `00-rl_guide.ipynb`


## PPO Example


In [2]:
from ray.rllib.algorithms.ppo import PPOConfig

# Configure PPO
config = (
    PPOConfig()
    .environment("CartPole-v1")
    .training(
        gamma=0.99,
        lr=5e-5,
        train_batch_size=4000,
    )
    .env_runners(num_env_runners=4)
)

# Build and train
algo = config.build_algo()
result = algo.train()
print(f"PPO - Reward: {result['env_runners']['episode_return_mean']:.2f}")

2025-10-19 23:17:31,437	WARNING algorithm_config.py:5050 -- You are running PPO on the new API stack! This is the new default behavior for this algorithm. If you don't want to use the new API stack, set `config.api_stack(enable_rl_module_and_learner=False,enable_env_runner_and_connector_v2=False)`. For a detailed migration guide, see here: https://docs.ray.io/en/master/rllib/new-api-stack-migration-guide.html


c:\Users\Maverick\Documents\GitHub\reinforcement\.venv\Lib\site-packages\ray\rllib\algorithms\algorithm.py:520: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
c:\Users\Maverick\Documents\GitHub\reinforcement\.venv\Lib\site-packages\ray\tune\logger\unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
The `JsonLogger interface is deprecated in favor of the `ray.tune.json.JsonLoggerCallback` interface and will be removed in Ray 2.7.
  self._loggers.append(cls(self.config, self.logdir, self.trial))
c:\Users\Maverick\Documents\GitHub\reinforcement\.venv\Lib\site-packages\ray\tune\logger\unified.py:53: RayDep

PPO - Reward: 20.06


## IMPALA Example (Windows-compatible)

In [1]:
from ray.rllib.algorithms.impala import IMPALAConfig
from tqdm import tqdm

# Configure IMPALA (local mode for Windows compatibility)
config = (
    IMPALAConfig()
    .environment("CartPole-v1")
    .learners(num_learners=0)  # Local mode (no distributed learners)
    .env_runners(num_env_runners=4)
    .training(
        gamma=0.99,
        lr=0.0005,
        train_batch_size=512,
    )
)

# Build and train for multiple iterations
algo = config.build_algo()
print("Training IMPALA on CartPole-v1...")

for i in tqdm(range(10), desc="Training", unit="iter"):
    result = algo.train()
    episodes = result['env_runners']['num_episodes']
    reward = result['env_runners']['episode_return_mean']
    length = result['env_runners']['episode_len_mean']
    tqdm.write(f"Iter {i+1:2d} | Episodes: {episodes:6.0f} | Reward: {reward:6.2f} | Length: {length:6.2f}")

2025-10-19 22:47:50,983	WARNING algorithm_config.py:5050 -- You are running IMPALA on the new API stack! This is the new default behavior for this algorithm. If you don't want to use the new API stack, set `config.api_stack(enable_rl_module_and_learner=False,enable_env_runner_and_connector_v2=False)`. For a detailed migration guide, see here: https://docs.ray.io/en/master/rllib/new-api-stack-migration-guide.html
c:\Users\Maverick\Documents\GitHub\reinforcement\.venv\Lib\site-packages\ray\rllib\algorithms\algorithm.py:520: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
c:\Users\Maverick\Documents\GitHub\reinforcement\.venv\Lib\site-packages\ray\tune\logger\unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray rel

Training IMPALA on CartPole-v1...


Training:  10%|█         | 1/10 [00:10<01:30, 10.08s/iter]

Iter  1 | Episodes:    504 | Reward:  29.76 | Length:  29.76


Training:  20%|██        | 2/10 [00:20<01:20, 10.09s/iter]

...

Training:  90%|█████████ | 9/10 [01:30<00:10, 10.11s/iter]

Iter  9 | Episodes:    318 | Reward:  67.52 | Length:  67.52


Training: 100%|██████████| 10/10 [01:41<00:00, 10.11s/iter]

Iter 10 | Episodes:    281 | Reward:  48.44 | Length:  48.44


(SingleAgentEnvRunner pid=8160) DeprecationWarning: `RLModule(config=[RLModuleConfig object])` has been deprecated. Use `RLModule(observation_space=.., action_space=.., inference_only=.., model_config=.., catalog_class=..)` instead. This will raise an error in the future!


In [18]:
# Save the trained model (Best Practices)
from pathlib import Path
from datetime import datetime
import json

# Define checkpoint directory with timestamp
repo_root = Path(__file__).parent.parent if '__file__' in locals() else Path.cwd().parent
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
checkpoint_base = repo_root / "checkpoints" / "impala_cartpole"
checkpoint_dir = checkpoint_base / timestamp
checkpoint_dir.mkdir(parents=True, exist_ok=True)

# Save training metadata
metadata = {
    "algorithm": "IMPALA",
    "environment": "CartPole-v1",
    "timestamp": timestamp,
    "training_iterations": 10,
    "final_reward_mean": float(result['env_runners']['episode_return_mean']),
    "final_episode_length": float(result['env_runners']['episode_len_mean']),
    "final_episodes": float(result['env_runners']['num_episodes']),
    "hyperparameters": {
        "gamma": 0.99,
        "lr": 0.0005,
        "train_batch_size": 512,
        "num_env_runners": 4,
        "num_learners": 0
    }
}

metadata_path = checkpoint_dir / "training_metadata.json"
with open(metadata_path, 'w') as f:
    json.dump(metadata, indent=2, fp=f)

# Save the model checkpoint
try:
    saved_path = algo.save(checkpoint_dir=str(checkpoint_dir))
    print(f"✓ Model saved successfully to: {saved_path}")
    print(f"✓ Metadata saved to: {metadata_path}")
    print(f"\nFinal Performance:")
    print(f"  - Mean Reward: {metadata['final_reward_mean']:.2f}")
    print(f"  - Mean Episode Length: {metadata['final_episode_length']:.2f}")
except Exception as e:
    print(f"✗ Error saving model: {e}")


✓ Model saved successfully to: TrainingResult(checkpoint=Checkpoint(filesystem=local, path=c:\Users\Maverick\Documents\GitHub\reinforcement\checkpoints\impala_cartpole\20251018_221334), metrics={'timers': {'training_iteration': 10.007739852955527, 'restore_env_runners': 2.6509628412545033e-05, 'training_step': 0.005670142667001269, 'sample': 0.0039115120421240055, 'learner_update_timer': 0.0013640342305409612, 'synch_weights': 0.004529510631270928}, 'env_runners': {'mean_num_episode_lists_received': 0.1516576123899905, 'num_episodes_lifetime': 4031.999999999998, 'episode_return_mean': 43.64, 'env_to_module_sum_episodes_length_in': 16.908273096856036, 'env_to_module_connector': {'timers': {'connectors': {'add_time_dim_to_batch_and_zero_pad': 1.0475611641219527e-05, 'numpy_to_tensor': 0.00014623494969214157, 'add_observations_from_episodes_to_batch': 2.7573970838038283e-05, 'add_states_from_episodes_to_batch': 5.1542618762668485e-06, 'batch_individual_items': 6.217203271734908e-05}}, 'co